In [1]:
import requests
from bs4 import BeautifulSoup
import re # a library to do regular expression matching
import pandas as pd
import numpy as np
import lxml

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [114]:
import warnings
warnings.filterwarnings('ignore')

In [203]:
page = requests.get("https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136374", timeout=15)
print(page)
print(page.text[:50])

<Response [200]>

<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
		
		<title>TOP 100 Swiss Startup Award Ranking 2011 - top100startup.ch</title>
		
		<meta name="Description" content="TOP 100 Swiss Startup Award Ranking 2011: The best Swiss Startups 2011 picked by a panel of 100 leading investors and startup experts." />				
	
		<meta name="Keywords" content="TOP 100, TOP 100 Swiss Startup Award, Best Swiss Startups, Venturelab, Swiss Startups, Innovation, Startups, TOP100SSU, Startup 


In [204]:
soup= BeautifulSoup(page.content, "html.parser")

In [205]:
soup2=soup.findAll("ul", {"class":"table-top100"})[0]

In [206]:
#Rank
soup2.findAll("span", {"class":"col-rank"})[1].text.split("Rank")[-1]  #here we have "Ranke" + "1"

'1'

In [207]:
#Name
soup2.findAll("span", {"class":"col-star","class":"text"})[0].text

'Optotune AG'

In [208]:
#Headquarter
soup2.findAll("span", {"class":"col-head"})[1].text.split("Headquarter")[-1]  #here we have "Headquarter" + "Zurich"

'Dietikon'

In [214]:
#Category
soup2.findAll("span", {"class":"col-cate"})[98].text.split("Category")[-1]  #here we have "Category" + "Fintech"

'Medtech'

In [194]:
#Description
soup2.findAll("span", {"class":"col-desc"})[1].text.strip().split("Description\n\n\r\n\t\t\t\t\t")[-1]

'Online marketplace for holiday rentals.'

In [195]:
#Link
soup2.findAll("a")[0].get("href")

'/HouseTrip'

In [115]:
def get_data(year,URL):  #get data for 2022
    page = requests.get(URL, timeout=15)
    soup= BeautifulSoup(page.content, "html.parser")
    soup2=soup.findAll("ul", {"class":"table-top100"})[0]
    
    cols = ["rank","name","headquarter","category","description","link","year"]
    df = pd.DataFrame(columns = cols)
    
    for i in range(100):
        rank = soup2.findAll("span", {"class":"col-rank"})[i+1].text.split("Rank")[-1]
        name = soup2.findAll("span", {"class":"col-star","class":"text"})[i].text
        headquarter = soup2.findAll("span", {"class":"col-head"})[i+1].text.split("Headquarter")[-1] 
        category = soup2.findAll("span", {"class":"col-cate"})[i+1].text.split("Category")[-1]
        description = soup2.findAll("span", {"class":"col-desc"})[i+1].text.strip().split("  ")[-1].strip(" \t")
        link = "https://www.top100startups.swiss"+ soup2.findAll("a")[i].get("href")
        year = year
        
        dict = { 
        'rank': rank,
        "name":name,
        "headquarter":headquarter,
        "category":category,
        "description":description,
        "link":link,
        "year":year}
        
        df = df.append(dict, ignore_index=True)
    return df

In [45]:
def get_data2(year,URL):  #get data for and before 2021 
    page = requests.get(URL, timeout=15)
    soup= BeautifulSoup(page.content, "html.parser")
    soup2=soup.findAll("ul", {"class":"table-top100"})[0]
    
    cols = ["rank","name","headquarter","category","description","link","year"]
    df = pd.DataFrame(columns = cols)
    
    for i in range(98): #99 for 2012, 98 for 2011
        rank = soup2.findAll("span", {"class":"col-rank"})[i+1].text.split("Rank")[-1]
        name = soup2.findAll("span", {"class":"col-star","class":"text"})[i].text
        headquarter = soup2.findAll("span", {"class":"col-head"})[i+1].text.split("Headquarter")[-1] 
        category = soup2.findAll("span", {"class":"col-cate"})[i+1].text.split("Category")[-1]
        description = soup2.findAll("span", {"class":"col-desc"})[i+1].text.strip().split("Description\n\n\r\n\t\t\t\t\t")[-1]
        link = "https://www.top100startups.swiss"+ soup2.findAll("a")[i].get("href")
        year = year
        
        dict = { 
        'rank': rank,
        "name":name,
        "headquarter":headquarter,
        "category":category,
        "description":description,
        "link":link,
        "year":year}
        
        df = df.append(dict, ignore_index=True)
    return df

In [216]:
df=get_data2(2011,"https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136374")

In [219]:
df.to_csv("2011_top_98_startups.csv")

In [249]:
df.reset_index()

,rank,name,headquarter,category,description,link,year
0,1,Optotune AG,Dietikon,Engineering,"Inexpensive, scalable, precisely focus tunable...",https://www.top100startups.swiss/Optotune,2011
1,2,Dacuda AG,Zürich,ICT,Scanning redefined,https://www.top100startups.swiss/Dacuda,2011
2,3,Doodle AG,Zürich,ICT,Easy scheduling,https://www.top100startups.swiss/Doodle,2011
3,4,Lemoptix SA,Lausanne,Engineering,Development of next generation optical MEMS (m...,https://www.top100startups.swiss/Lemoptix,2011
4,5,Poken SA (acquired by GES),Lausanne,ICT,Collect people and things with a touch,https://www.top100startups.swiss/Poken,2011
5,6,InSphero AG,Schlieren,Biotech,Market leader in 3D cell culture,https://www.top100startups.swiss/InSphero,2011
6,7,Finity SA (former paper.li),Lausanne,ICT,Create your newspaper,https://www.top100startups.swiss/Smallrivers,2011
7,8,HouseTrip SA,London,ICT,Online marketplace for holiday rentals.,https://www.top100startups.swiss/HouseTrip,2011
8,9,Malcisbo AG,Schlieren,Biotech,Platform to identify carbohydrate targets in p...,https://www.top100startups.swiss/Malcisbo,2011
9,10,Aïmago SA (acquired by Novadaq),Lausanne,Medtech,Taking the Guesswork out of Tissue Perfusion A...,https://www.top100startups.swiss/Aimago,2011


In [226]:
for y in range(2011,2023):
    print(y)
    df_y=pd.read_csv("../data/"+(str(y)+"_top_100_startups.csv"))

In [238]:
df_2011 = pd.read_csv("../data/2011_top_100_startups.csv")
df_2012 = pd.read_csv("../data/2012_top_100_startups.csv")
df_2013 = pd.read_csv("../data/2013_top_100_startups.csv")
df_2014 = pd.read_csv("../data/2014_top_100_startups.csv")
df_2015 = pd.read_csv("../data/2015_top_100_startups.csv")
df_2016 = pd.read_csv("../data/2016_top_100_startups.csv")
df_2017 = pd.read_csv("../data/2017_top_100_startups.csv")
df_2018 = pd.read_csv("../data/2018_top_100_startups.csv")
df_2019 = pd.read_csv("../data/2019_top_100_startups.csv")
df_2020 = pd.read_csv("../data/2020_top_100_startups.csv")
df_2021 = pd.read_csv("../data/2021_top_100_startups.csv")
df_2022 = pd.read_csv("../data/2022_top_100_startups.csv")

In [239]:
df_list=[df_2022,df_2021,df_2020,df_2019,df_2018,df_2017,df_2016,df_2015,df_2014,df_2013,df_2012,df_2011]

In [246]:
big_df

,index,Unnamed: 0,rank,name,headquarter,category,description,link,year
0,0,0,1,Yokoy Group AG,Zurich,Fintech,Your expenses and company credit cards on auto...,https://www.top100startups.swiss/yokoy,2022
1,1,1,2,Planted Foods AG,Kemptthal,Foodtech,Planted meats,https://www.top100startups.swiss/Planted,2022
2,2,2,3,Ledgy AG,Zurich,Fintech,Your equity set for growth,https://www.top100startups.swiss/Ledgy,2022
3,3,3,4,CUTISS AG,Schlieren,Biotech,Personalized skin,https://www.top100startups.swiss/CUTISS,2022
4,4,4,5,CREAL SA,Ecublens,Engineering,Enabling Augmented Reality within arm's reach,https://www.top100startups.swiss/CREAL,2022
5,5,5,6,9T Labs AG,Zürich,Engineering,3D carbon fiber printing technology,https://www.top100startups.swiss/9T,2022
6,6,6,7,Carvolution AG,Bannwil,ICT,Car Subscription Model for Switzerland,https://www.top100startups.swiss/Carvolution,2022
7,7,7,8,HAYA Therapeutics SA,Epalinges,Biotech,Treating heart failure through innovative RNA-...,https://www.top100startups.swiss/HAYA,2022
8,8,8,9,Bloom Biorenewables SA,Marly,Cleantech,Making biomass a true alternative to petroleum,https://www.top100startups.swiss/Bloom,2022
9,9,9,10,Daphne Technology SA,St-Sulpice VD,Cleantech,Climate tech comp reducing toxic pollutants & ...,https://www.top100startups.swiss/Daphne,2022


In [252]:
#big_df.to_csv("all_top_100_startups.csv")

In [3]:
big_df=pd.read_csv("../data/all_top_100_startups.csv")

In [4]:
big_df.head(2)

,Unnamed: 0.1,Unnamed: 0,rank,name,headquarter,category,description,link,year
0,0,0,1,Yokoy Group AG,Zurich,Fintech,Your expenses and company credit cards on auto...,https://www.top100startups.swiss/yokoy,2022
1,1,1,2,Planted Foods AG,Kemptthal,Foodtech,Planted meats,https://www.top100startups.swiss/Planted,2022


In [5]:
big_df=big_df.drop(["Unnamed: 0.1","Unnamed: 0"],axis=1)

In [6]:
big_df.head(3)

,rank,name,headquarter,category,description,link,year
0,1,Yokoy Group AG,Zurich,Fintech,Your expenses and company credit cards on auto...,https://www.top100startups.swiss/yokoy,2022
1,2,Planted Foods AG,Kemptthal,Foodtech,Planted meats,https://www.top100startups.swiss/Planted,2022
2,3,Ledgy AG,Zurich,Fintech,Your equity set for growth,https://www.top100startups.swiss/Ledgy,2022


In [8]:
big_df.groupby(["category","year"]).name.count()

category     year
Biotech      2011    17
             2012    16
             2013    13
             2014    17
             2015    15
             2016    19
             2017    24
             2018    17
             2019    18
             2020    21
             2021    17
             2022    19
Cleantech    2011     5
             2012    10
             2013    12
             2014     9
             2015     8
             2016     8
             2017     7
             2018     6
             2019     4
             2020     4
             2021     6
             2022     9
Engineering  2011    19
             2012    21
             2013    21
             2014    16
             2015    15
             2016     9
             2017     7
             2018    12
             2019    10
             2020     7
             2021    12
             2022     9
Fintech      2011     2
             2012     1
             2013     1
             2014     1
             2015     

## Expanding current big dataframe

In [9]:
big_df.link.iloc[-1]

'https://www.top100startups.swiss/Pearltec'

In [10]:
page = requests.get(big_df.link.iloc[-1], timeout=15)
#print(page)
#print(page.text[:50])

In [11]:
soup= BeautifulSoup(page.content, "html.parser")

In [78]:
#name
soup.findAll("h1")[0].text

'Pearl Technology AG'

In [12]:
len(soup.findAll("div", {"class":"categories-container"})[0].findAll("a"))

4

In [74]:
#categories
print(len(soup.findAll("div", {"class":"categories-container"})[0].findAll("a")))
soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[3].text #len = 4

4


'Imaging and Radiology'

In [143]:
#incorporated
soup.findAll("div", {"class":"info"})[0].find("span", {"class":"text"}).text

'October 2016'

In [144]:
# long description
re.sub('\r','', soup.findAll("div", {"class":"text-container"})[0].text.strip())

'Pearl Technology AG develops medical devices for Computed Tomography and Magnetic Resonance Imaging that allows a higher patient throughput, increased image quality and improved patient comfort.'

In [145]:
# Website
soup.findAll("div", {"class":"social-media"})[0].findAll("a", {"class":"icon-website"})[0].get("href")

'http://www.pearl-technology.ch'

## Adding new data to existing Dataframe

In [17]:
big_df.link.nunique()

510

In [32]:
df_subset=big_df[~big_df.link.str.contains("index")].copy()
df_subset

,rank,name,headquarter,category,description,link,year
0,1,Yokoy Group AG,Zurich,Fintech,Your expenses and company credit cards on auto...,https://www.top100startups.swiss/yokoy,2022
1,2,Planted Foods AG,Kemptthal,Foodtech,Planted meats,https://www.top100startups.swiss/Planted,2022
2,3,Ledgy AG,Zurich,Fintech,Your equity set for growth,https://www.top100startups.swiss/Ledgy,2022
3,4,CUTISS AG,Schlieren,Biotech,Personalized skin,https://www.top100startups.swiss/CUTISS,2022
4,5,CREAL SA,Ecublens,Engineering,Enabling Augmented Reality within arm's reach,https://www.top100startups.swiss/CREAL,2022
5,6,9T Labs AG,Zürich,Engineering,3D carbon fiber printing technology,https://www.top100startups.swiss/9T,2022
6,7,Carvolution AG,Bannwil,ICT,Car Subscription Model for Switzerland,https://www.top100startups.swiss/Carvolution,2022
7,8,HAYA Therapeutics SA,Epalinges,Biotech,Treating heart failure through innovative RNA-...,https://www.top100startups.swiss/HAYA,2022
8,9,Bloom Biorenewables SA,Marly,Cleantech,Making biomass a true alternative to petroleum,https://www.top100startups.swiss/Bloom,2022
9,10,Daphne Technology SA,St-Sulpice VD,Cleantech,Climate tech comp reducing toxic pollutants & ...,https://www.top100startups.swiss/Daphne,2022


In [19]:
df_subset.link.iloc[0]

'https://www.top100startups.swiss/yokoy'

In [26]:
test=[df_subset.link.iloc[0],df_subset.link.iloc[1]]
test

['https://www.top100startups.swiss/yokoy',
 'https://www.top100startups.swiss/Planted']

In [26]:
for a, b in enumerate(test):
    print ("a ="+str(a))
    print ("b ="+str(b))

a =0
b =https://www.top100startups.swiss/yokoy
a =1
b =https://www.top100startups.swiss/Planted


In [30]:
df_subset.link.unique()[290:300]

array(['https://www.top100startups.swiss/MOVU',
       'https://www.top100startups.swiss/PIQUR',
       'https://www.top100startups.swiss/ecoRobotix',
       'https://www.top100startups.swiss/SWISSto12',
       'https://www.top100startups.swiss/FEMTOprint',
       'https://www.top100startups.swiss/Memo',
       'https://www.top100startups.swiss/Fastree3D',
       'https://www.top100startups.swiss/DomoHealth',
       'https://www.top100startups.swiss/NEEO',
       'https://www.top100startups.swiss/Geosatis'], dtype=object)

In [113]:
def more_data(series):
    cols = []
    df = pd.DataFrame(columns = cols)
    for num, link in enumerate(series):
        print(num)
        page = requests.get(link, timeout=15)
        soup= BeautifulSoup(page.content, "html.parser")
        
        #leng=len(soup.findAll("div", {"class":"categories-container"})[0].findAll("a"))
        try:
            name=soup.findAll("h1")[0].text
            #print(name)
            cat1=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[0].text
            try:
                cat2=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[1].text
            except IndexError:
                cat2=""
            try:
                cat3=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[2].text
            except IndexError:
                cat3=""
            try:
                cat4=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[3].text
            except IndexError:
                cat4=""
            try:    
                cat5=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[4].text
            except IndexError:
                cat5=""        
            try:
                cat6=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[5].text
            except IndexError:
                cat6=""
            try:
                cat7=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[6].text
            except IndexError:
                cat7=""
            try:
                cat8=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[7].text
            except IndexError:
                cat8=""
            try:
                cat9=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[8].text
            except IndexError:
                cat9=""
            try:
                cat10=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[9].text
            except IndexError:
                cat10=""            
            try:
                cat11=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[10].text
            except IndexError:
                cat11=""                
            try:
                cat12=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[11].text
            except IndexError:
                cat12=""
            try:
                cat13=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[12].text
            except IndexError:
                cat13=""
            try:
                cat14=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[13].text
            except IndexError:
                cat14=""
            try:
                cat15=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[14].text
            except IndexError:
                cat15=""                
            try:
                cat16=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[15].text
            except IndexError:
                cat16=""
            try:
                cat17=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[16].text
            except IndexError:
                cat17=""         
            try:
                cat18=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[17].text
            except IndexError:
                cat18=""    
            try:
                cat19=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[18].text
            except IndexError:
                cat19=""    
            try:
                cat20=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[19].text
            except IndexError:
                cat20=""    
            try:
                cat21=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[20].text
            except IndexError:
                cat21=""                    
                
            inc=soup.findAll("div", {"class":"info"})[0].find("span", {"class":"text"}).text
            desc=re.sub('\r','', soup.findAll("div", {"class":"text-container"})[0].text.strip())
            try:
                web=soup.findAll("div", {"class":"social-media"})[0].findAll("a", {"class":"icon-website"})[0].get("href")
            except IndexError:
                web=""
            link=link

            dicti = { 
            "name":name,
            'category_1':cat1,
            "category_2":cat2,
            "category_3":cat3,
            "category_4":cat4,
            "category_5":cat5,
            "category_6":cat6,
            "category_7":cat7,
            "category_8":cat8,
            "category_9":cat9,
            "category_10":cat10,
            "category_11":cat11,
            "category_12":cat12,
            "category_13":cat13,
            "category_14":cat14,
            "category_15":cat15,
            "category_16":cat16,
            "category_17":cat17,
            "category_18":cat18,
            "category_19":cat19,
            "category_20":cat20,
            "category_21":cat21,                
            "incorporated":inc,
            "long description":desc,
            "company_website":web,
            "link":link}

            #print(dicti)
            new_df=pd.DataFrame(dicti, index=[0])
            #df = df.append(dicti, ignore_index=True)
            df = pd.concat([df, new_df], ignore_index=True)
            
        except IndexError:
            print("Hello World -------------------------------------")
    return df

In [112]:
df_subset.link.unique()[0]

'https://www.top100startups.swiss/yokoy'

In [114]:
all_category_frame=more_data(big_url_array)

0
1
Hello World -------------------------------------
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


In [115]:
all_category_frame.to_csv("startup_detail3.csv")

In [34]:
for num, link in enumerate(test):
    page = requests.get(link, timeout=15)
    soup= BeautifulSoup(page.content, "html.parser")
    leng=len(soup.findAll("div", {"class":"categories-container"})[0].findAll("a"))
    print(leng)
    df_subset["category_1"].iloc[num]=""
    df_subset["category_2"].iloc[num]=""
    df_subset["category_3"]=""
    df_subset["category_4"]=""
    df_subset["category_5"]=""
    df_subset["category_6"]=""
    if leng > 0:
        df_subset["category_1"].iloc[num]=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[0].text
    if leng > 1:
        df_subset["category_1"].iloc[num]=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[1].text
    if leng > 2:
        df_subset["category_1"]=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[2].text 
    if leng > 3:
        df_subset["category_1"]=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[3].text 
    if leng > 4:
        df_subset["category_1"]=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[4].text 
    if leng > 5:
        df_subset["category_1"]=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[5].text
    if leng > 6:
        df_subset["category_1"]=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[6].text 
    if leng > 7:
        df_subset["category_1"]=soup.findAll("div", {"class":"categories-container"})[0].findAll("a")[7].text
    

6


KeyError: 'category_1'

In [100]:
df_subset

,rank,name,headquarter,category,description,link,year
0,1,Yokoy Group AG,Zurich,Fintech,Your expenses and company credit cards on auto...,https://www.top100startups.swiss/yokoy,2022
1,2,Planted Foods AG,Kemptthal,Foodtech,Planted meats,https://www.top100startups.swiss/Planted,2022
2,3,Ledgy AG,Zurich,Fintech,Your equity set for growth,https://www.top100startups.swiss/Ledgy,2022
3,4,CUTISS AG,Schlieren,Biotech,Personalized skin,https://www.top100startups.swiss/CUTISS,2022
4,5,CREAL SA,Ecublens,Engineering,Enabling Augmented Reality within arm's reach,https://www.top100startups.swiss/CREAL,2022
5,6,9T Labs AG,Zürich,Engineering,3D carbon fiber printing technology,https://www.top100startups.swiss/9T,2022
6,7,Carvolution AG,Bannwil,ICT,Car Subscription Model for Switzerland,https://www.top100startups.swiss/Carvolution,2022
7,8,HAYA Therapeutics SA,Epalinges,Biotech,Treating heart failure through innovative RNA-...,https://www.top100startups.swiss/HAYA,2022
8,9,Bloom Biorenewables SA,Marly,Cleantech,Making biomass a true alternative to petroleum,https://www.top100startups.swiss/Bloom,2022
9,10,Daphne Technology SA,St-Sulpice VD,Cleantech,Climate tech comp reducing toxic pollutants & ...,https://www.top100startups.swiss/Daphne,2022


In [172]:
big_df[big_df.link in "index" ]

TypeError: 'in <string>' requires string as left operand, not Series

In [39]:
big_df[~big_df.link.str.contains("index")].sort_values(by="name",ascending = True)

,rank,name,headquarter,category,description,link,year
563,64,1Drop Diagnostics Sàrl,Neuchâtel,Medtech,Saving lives through better and faster portabl...,https://www.top100startups.swiss/1Drop,2017
747,48,1Drop Diagnostics Sàrl,Neuchâtel,Medtech,Saving lives through better and faster portabl...,https://www.top100startups.swiss/1Drop,2015
825,26,1Drop Diagnostics Sàrl,Neuchâtel,Medtech,Saving lives through better and faster portabl...,https://www.top100startups.swiss/1Drop,2014
884,85,3BaysOver,Lausanne,ICT,The Professional Network for Travel.,https://www.top100startups.swiss/3BaysOver,2014
734,35,3BaysOver,Lausanne,ICT,The Professional Network for Travel.,https://www.top100startups.swiss/3BaysOver,2015
1058,60,42matters AG,Zurich,ICT,App Intelligence as SaaS,https://www.top100startups.swiss/42matters,2012
1191,94,42matters AG,Zurich,ICT,App Intelligence as SaaS,https://www.top100startups.swiss/42matters,2011
988,89,42matters AG,Zurich,ICT,App Intelligence as SaaS,https://www.top100startups.swiss/42matters,2013
102,3,9T Labs AG,Zürich,Engineering,3D carbon fiber printing technology,https://www.top100startups.swiss/9T,2021
425,26,9T Labs AG,Zürich,Engineering,3D carbon fiber printing technology,https://www.top100startups.swiss/9T,2018


In [46]:
test=big_df[big_df.link.str.contains("index")].link.unique()
test

array(['https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=18261',
       'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=31198',
       'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=7520',
       'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=7498',
       'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=8824',
       'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=5376',
       'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b2

In [47]:
test=list(test)
test

['https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=18261',
 'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=31198',
 'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=7520',
 'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=7498',
 'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=8824',
 'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=5376',
 'https://www.top100startups.swissindex.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33A

In [50]:
for i,en in enumerate(test):
    test[i]=re.sub('https://www.top100startups.swiss','https://www.top100startups.swiss/',en)
test

['https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=18261',
 'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=31198',
 'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=7520',
 'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=7498',
 'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=8824',
 'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=5376',
 'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-7

In [53]:
test=np.array(test)
test

array(['https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=18261',
       'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=31198',
       'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=7520',
       'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=7498',
       'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=8824',
       'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136348&profil_id=5376',
       'https://www.top100startups.swiss/index.cfm?CFID=72313628&CFT

In [63]:
type(test)

numpy.ndarray

In [62]:
type(df_subset.link.unique())

numpy.ndarray

In [67]:
big_url_array=np.concatenate((test,df_subset.link.unique()),axis=None)

510

510